In [2]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("batubayk/TR-News")

In [24]:
train_ds = ds['train']


# DataFrame'e çevirme
dataset_new = pd.DataFrame(train_ds)

# İlk satıra erişim (iloc ile)
print(dataset_new.iloc[0])


abstract    Şarkıcı Tuğba Özerk, annesine canlı yayında yö...
author                                                   None
content     Tuğba Özerk, “Duymayan Kalmasın” programında, ...
date                                       13.01.2017 - 12:38
source                                              haberturk
tags                                                       []
title       Tuğba Özerk'ten Deniz Akkaya’ya 100 bin TL’lik...
topic                                                  Fiskos
url         https://www.haberturk.com/magazin/fiskos/haber...
Name: 0, dtype: object


In [1]:
import torch
import re
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pytorch_lightning as pl
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download('omw-1.4')
import os
import string
import torch.nn as nn
import pandas as pd
import numpy as np
import evaluate
import matplotlib.pyplot as plt
import transformers
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import get_linear_schedule_with_warmup, AdamW
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Burak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Burak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Burak\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [20]:
dataset_new.head()
len(dataset_new.iloc[8].content)

2026

In [21]:
MODEL = T5ForConditionalGeneration.from_pretrained("t5-large", return_dict=True)
TOKENIZER = T5Tokenizer.from_pretrained("t5-large")
BATCH_SIZE = 1
TEXT_LEN = 2048
HEADLINE_LEN = 64
EPOCHS = 3
# GPU kontrolü
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Training on: {DEVICE}")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Training on: cuda:0


In [44]:
class NewsSummaryDataset(Dataset):
    def __init__(self, df, tokenizer, text_len, headline_len):
        self.dataset_new= dataset_new
        self.headlines = self.dataset_new["abstract"]
        self.text = self.dataset_new["content"]
        self.tokenizer = tokenizer
        self.text_len = text_len
        self.headline_len = headline_len
  
    def __len__(self):
        return len(self.headlines)
  
    def __getitem__(self, idx):
        # T5 transformers performs different tasks by prepending the particular prefix to the input text.
        text = "summarize:" + str(self.text[idx])                # In order to avoid dtype mismatch, as T5 is text-to-text transformer, the datatype must be string
        headline = str(self.headlines[idx])

        text_tokenizer = self.tokenizer(text, max_length=self.text_len, padding="max_length",
                                                        truncation=True, add_special_tokens=True)
        headline_tokenizer = self.tokenizer(headline, max_length=self.headline_len, padding="max_length",
                                                        truncation=True, add_special_tokens=True)
        return {
            "input_ids": torch.tensor(text_tokenizer["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(text_tokenizer["attention_mask"], dtype=torch.long),
            "summary_ids": torch.tensor(headline_tokenizer["input_ids"], dtype=torch.long),
            "summary_mask": torch.tensor(headline_tokenizer["attention_mask"], dtype=torch.long)
        }

In [45]:
class NewsSummaryDataModule(pl.LightningDataModule):
    def __init__(self, 
                train_df, 
                val_df,
                test_df,
                batch_size,
                tokenizer,
                text_len,
                summary_len):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.batch_size = batch_size
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.text_len = text_len
        self.summary_len = summary_len
    
    def setup(self, stage=None):
        self.train_dataset = NewsSummaryDataset(
            self.train_df,
            self.tokenizer,
            self.text_len,
            self.summary_len)
        
        self.val_dataset = NewsSummaryDataset(
            self.val_df,
            self.tokenizer,
            self.text_len,
            self.summary_len)
        
        self.test_dataset = NewsSummaryDataset(
            self.test_df,
            self.tokenizer,
            self.text_len,
            self.summary_len
        )
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=4)
        
    def test_dataloader(self):
        return DataLoader(
              self.test_dataset,
              batch_size=self.batch_size,
              num_workers=4
          )

In [46]:
train_df, test_df = train_test_split(dataset_new, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [47]:
news_summary_module = NewsSummaryDataModule(train_df, val_df, test_df, BATCH_SIZE, TOKENIZER, TEXT_LEN, HEADLINE_LEN)
news_summary_module.setup()

In [48]:
class NewsSummaryModel(pl.LightningModule):
    def __init__(self):
        super(NewsSummaryModel, self).__init__()
        self.model = MODEL
  
    def forward(self, input_ids, attention_mask, labels=None, decoder_attention_mask=None):
        outputs = self.model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels,
                        decoder_attention_mask=decoder_attention_mask)
        return outputs.loss, outputs.logits
  
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["summary_ids"]
        decoder_attention_mask = batch["summary_mask"]

        loss, output = self(input_ids, attention_mask, labels, decoder_attention_mask)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["summary_ids"]
        decoder_attention_mask = batch["summary_mask"]

        loss, output = self(input_ids, attention_mask, labels, decoder_attention_mask)
        return loss
 
    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        loss, output = self(input_ids=input_ids, 
                      attention_mask=attention_mask)
        return loss
    
    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=0.0001)
        scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=0,
                num_training_steps=EPOCHS*len(dataset_new))
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

In [49]:
model = NewsSummaryModel()

In [ ]:
trainer = pl.Trainer(
    max_epochs=EPOCHS,
    accelerator="gpu",
    devices=[0]  # GPU 0 ve 1'i kullanmak için
)

trainer.fit(model, news_summary_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params | Mode
------------------------------------------------------------
0 | model | T5ForConditionalGeneration | 737 M  | eval
------------------------------------------------------------
737 M     Trainable params
0         Non-trainable params
737 M     Total params
2,950.672 Total estimated model params size (MB)
0         Modules in train mode
1069      Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]